# Imports & setup

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import numpy as np
import pickle

In [3]:
features = ['Was_home',
 'Team_rating',
 'Opp_rating',
 'Rating_difference',
 'Min_2',
 'Start_2',
 'Gls_2',
 'Sh_2',
 'SoT_2',
 'Ast_2',
 'CrdY_2',
 'CrdR_2',
 'xG_2',
 'xA_2',
 'Team_CS_2',
 'Team_score_2',
 'Opp_score_2',
 'Team_result_2',
 'ICT_index_2',
 'FPL_points_2',
 'Baseline_points_2',
 'Bonus_2',
 'Min_4',
 'Start_4',
 'Gls_4',
 'Sh_4',
 'SoT_4',
 'Ast_4',
 'CrdY_4',
 'CrdR_4',
 'xG_4',
 'xA_4',
 'Team_CS_4',
 'Team_score_4',
 'Opp_score_4',
 'Team_result_4',
 'ICT_index_4',
 'FPL_points_4',
 'Baseline_points_4',
 'Bonus_4',
 'Min_30',
 'Start_30',
 'Gls_30',
 'Sh_30',
 'SoT_30',
 'Ast_30',
 'CrdY_30',
 'CrdR_30',
 'xG_30',
 'xA_30',
 'Team_CS_30',
 'Team_score_30',
 'Opp_score_30',
 'Team_result_30',
 'ICT_index_30',
 'FPL_points_30',
 'Baseline_points_30',
 'Bonus_30',
 'DEF',
 'FWD',
 'MID']

info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating", "FPL_pos", "Price"]
to_predict = ["FPL_points"]

In [4]:
len(features)

61

# Reading from files

In [5]:
df = pd.read_csv("data/upcoming/upcoming_fixtures.csv")
CURRENT_SEASON = "2023-24"

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (71) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
X_test = df[df["Season"] == CURRENT_SEASON][features]
# .reset_index(drop=True)

In [7]:
df.columns

Index(['Unnamed: 0', 'Ast_2', 'Ast_30', 'Ast_4', 'Baseline_points_2',
       'Baseline_points_30', 'Baseline_points_4', 'Bonus_2', 'Bonus_30',
       'Bonus_4', 'CrdR_2', 'CrdR_30', 'CrdR_4', 'CrdY_2', 'CrdY_30', 'CrdY_4',
       'DEF', 'FPL_points', 'FPL_points_2', 'FPL_points_30', 'FPL_points_4',
       'FPL_pos', 'FWD', 'GKP', 'GW', 'Gls_2', 'Gls_30', 'Gls_4',
       'ICT_index_2', 'ICT_index_30', 'ICT_index_4', 'MID', 'Min_2', 'Min_30',
       'Min_4', 'Name_original', 'Opp_rating', 'Opp_score_2', 'Opp_score_30',
       'Opp_score_4', 'Opponent', 'Rating_difference', 'Season', 'Sh_2',
       'Sh_30', 'Sh_4', 'SoT_2', 'SoT_30', 'SoT_4', 'Start_2', 'Start_30',
       'Start_4', 'Team', 'Team_CS_2', 'Team_CS_30', 'Team_CS_4',
       'Team_rating', 'Team_result_2', 'Team_result_30', 'Team_result_4',
       'Team_score_2', 'Team_score_30', 'Team_score_4', 'Was_home', 'xA_2',
       'xA_30', 'xA_4', 'xG_2', 'xG_30', 'xG_4', 'Finished', 'Date', 'Price'],
      dtype='object')

In [8]:
X_test.columns

Index(['Was_home', 'Team_rating', 'Opp_rating', 'Rating_difference', 'Min_2',
       'Start_2', 'Gls_2', 'Sh_2', 'SoT_2', 'Ast_2', 'CrdY_2', 'CrdR_2',
       'xG_2', 'xA_2', 'Team_CS_2', 'Team_score_2', 'Opp_score_2',
       'Team_result_2', 'ICT_index_2', 'FPL_points_2', 'Baseline_points_2',
       'Bonus_2', 'Min_4', 'Start_4', 'Gls_4', 'Sh_4', 'SoT_4', 'Ast_4',
       'CrdY_4', 'CrdR_4', 'xG_4', 'xA_4', 'Team_CS_4', 'Team_score_4',
       'Opp_score_4', 'Team_result_4', 'ICT_index_4', 'FPL_points_4',
       'Baseline_points_4', 'Bonus_4', 'Min_30', 'Start_30', 'Gls_30', 'Sh_30',
       'SoT_30', 'Ast_30', 'CrdY_30', 'CrdR_30', 'xG_30', 'xA_30',
       'Team_CS_30', 'Team_score_30', 'Opp_score_30', 'Team_result_30',
       'ICT_index_30', 'FPL_points_30', 'Baseline_points_30', 'Bonus_30',
       'DEF', 'FWD', 'MID'],
      dtype='object')

In [9]:
file = open("models/GBR.pkl",'rb')
model = pickle.load(file)

In [10]:
model

GradientBoostingRegressor(random_state=42)

# Predicting

In [11]:
def adjust_for_injuries(df):
    active_players = pd.read_csv("data/misc/active_players.csv")
    injured_players = active_players[active_players["chance_of_playing_this_round"] == 0]["name"].to_list()
    df.loc[df["Name_original"].isin(injured_players),"Pred"] = 0
    return df

In [12]:
def get_predictions(model, df, X, all_remaining=False):
    # make predictions on the test data and glues them to the rest of the dataframe
    predictions = model.predict(X)
    df_predictions = df[(df["Season"] == CURRENT_SEASON)].reset_index(drop=True)
        
    print(len(predictions), df_predictions.shape)
        
    df_predictions.loc[:, "Pred"] = predictions
    preds = df_predictions[info + ["Pred"]]
    
    preds = adjust_for_injuries(preds).sort_values(by=["Pred"], ascending = False)
    
    return preds

In [13]:
preds = get_predictions(model, df, X_test)

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- GKP

  warnings.warn(message, FutureWarning)


ValueError: X has 61 features, but GradientBoostingRegressor is expecting 62 features as input.

In [ ]:
# preds[(preds["FPL_pos"] == "GK") & (preds["GW"] == 2)].head(30)

In [ ]:
# next gameweek
preds[preds["GW"] == 2].head(20)

In [ ]:
# all preds
preds.head(20)

In [ ]:
# points all season
pd.DataFrame(preds.groupby(["Name_original"])["Pred"].sum().sort_values(ascending=False)).head(30)

# Saving to file

In [ ]:
preds.to_csv("predictions/preds_next_season.csv", index=False)